<h1><center>Laboratorio 6: Optimización de modelos 🧪</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Josefa Anselmo.
- Nombre de alumno 2: Tamara Carrasco.


Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### **Link de repositorio de GitHub:** [TamoJosha](https://github.com/Tamaracarrasco/TamoJosha-Lab-de-Prog-MDS-Prim2025)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.


### Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("sales.csv")

df.head(10)

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696
5,5,31/01/12,Athens,37.97945,23.71622,672130,shop_1,orange-power,glass,500ml,1.00,15041
6,6,31/01/12,Athens,37.97945,23.71622,672130,shop_1,orange-power,can,330ml,0.43,34578
7,7,31/01/12,Athens,37.97945,23.71622,672130,shop_1,gazoza,glass,500ml,0.49,44734
8,8,31/01/12,Athens,37.97945,23.71622,672130,shop_1,lemon-boost,glass,500ml,0.70,18623
9,9,31/01/12,Athens,37.97945,23.71622,672130,shop_1,lemon-boost,plastic,1.5lt,2.21,9645


In [3]:
## Exploración básica de los datos

print("\nDimensiones del dataset:\n", df.shape)
print("\n Tipo de valores en las columnas: \n", df.info())
print("\n Cantidad de valores nulos: \n", df.isna().sum())
print("\n Valores duplicados: \n", df.duplicated().sum())
print("\n describe de las variables númericas: \n", df.select_dtypes(include="number").describe())
print("\n Estadísticas de las variables no numéricas: \n", df.select_dtypes(include="object").describe())

## Rango temporal de los datos.

print("\n ========= RANGO TEMPORAL (Aparente) =========")
print("Fecha inicial: ", df["date"].min())
print("última fecha: ", df["date"].max())


Dimensiones del dataset:
 (7456, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7456 entries, 0 to 7455
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         7456 non-null   int64  
 1   date       7456 non-null   object 
 2   city       7456 non-null   object 
 3   lat        7456 non-null   float64
 4   long       7456 non-null   float64
 5   pop        7456 non-null   int64  
 6   shop       7456 non-null   object 
 7   brand      7456 non-null   object 
 8   container  7456 non-null   object 
 9   capacity   7456 non-null   object 
 10  price      7456 non-null   float64
 11  quantity   7456 non-null   int64  
dtypes: float64(3), int64(3), object(6)
memory usage: 699.1+ KB

 Tipo de valores en las columnas: 
 None

 Cantidad de valores nulos: 
 id           0
date         0
city         0
lat          0
long         0
pop          0
shop         0
brand        0
container    0
capacity     0
price   

**Comentario**

 De las estadísticas, se desprende lo siguiente:
- NO hay valores nulos.
- No hay valores duplicados.
- Al parecer no hay valores raros como cantidades de "quantity" negativas.
- dentro de las variables categóricas, ``capacity`` es una variable categórica ordinal. El resto de variables categóricas son nominales.
- Se tiene ``date``, serie de tiempo.


- Se tienen 5 ciudades:  Athens, Thessaloniki, Patra, Larisa, Irakleion.
- Hay 6 tiendas distintas: shop_1,..., shop_6.
	* shop_1 solo está en Athens.
	* shop_2 solo está en Irakleion.
	* shop_3 también está en Athens.
	* shop_4 está en Thessaloniki.
	* shop_5 está en Larisa.
	* shop_6 está en Patra.

- Se tienen 5 marcas posibles:
	* kinder-cola: plastic, can, glass
	* adult-cola: plastic, can, glass
	* lemon-boost: glass, can, plastic
	* gazoza: glass, plastic, can
	* orange-power: glass, can, plastic
    	
- hay 3 tipos de envase (container):
	- glass: hay de capacidad de 500ml y 330 ml
	- can:  tiene solo capacidad de 330ml
	- plastic: tiene solo de 1.5lts

**La variable target es ``quantity`**

## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [4]:
%!pip install autopep8

UsageError: Line magic function `%!pip` not found.


In [5]:
# importación de librerías importantes.

# separacion de conjuntos
from sklearn.model_selection import train_test_split

# para el pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from xgboost import XGBRegressor

# métricas

from sklearn.metrics import mean_absolute_error as MAE

# para guardar los modelos con pickle
import os
import pickle

from sklearn import set_config
set_config(transform_output="pandas")

# Inserte su código acá

# Definición de FunctionTraformer (Punto 2)


def extrae_fecha(dataframe_in, col="date"):
    """
    Esta función debería estraer el día, mes y año
    de la variable "date" de un dataframe.
    Guarda las variables en el formato categórico
    de pandas.

    Parámetros:
    -------------------
    dataframe_in: dataframe para extraer los datos. 
    Debe contener columna "date".

    col: columna de la fecha. Default "date".

    """
    # primer transformamos esa columna a una nueva

    df_copy = dataframe_in.copy()

    df_copy["fecha"] = pd.to_datetime(
        df_copy[col], dayfirst=True, errors="coerce", format="%d/%m/%y")

    # se crean las columnas de día, mes y año.

    df_copy["day"] = df_copy["fecha"].dt.day.astype("category")
    df_copy["month"] = df_copy["fecha"].dt.month.astype("category")
    df_copy["year"] = df_copy["fecha"].dt.year.astype("category")

    return df_copy


date_transformer = FunctionTransformer(extrae_fecha)


# SEPARACIÓN DE CONJUNTOS (Punto 1)

df_x = df.drop(columns="quantity").copy()
df_y = df["quantity"]


# fijo el random state por si acaso.
X_train, X_val, y_train, y_val = train_test_split(
    df_x, df_y, random_state=42, shuffle=False, stratify=None, train_size=0.7)

X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, test_size=0.33, shuffle=False, stratify=None, random_state=42)


drop_cols = ["id"]

num_cols = ["lat", "long", "pop", "price"]
cat_cols = ["city", "shop", "brand", "container",
            "capacity", "day", "month", "year"]

# PIPELINES POR GRUPO (punto 3)

num_std_pipeline = Pipeline(
    [
        # aunque no hayna nulos, coloco esto por completitud
        ("imp", SimpleImputer(strategy="median")),
        ("sc", StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    [
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(
            handle_unknown="ignore", sparse_output=False
        ))
    ]
)

# COLUMN TRANSFORMER (PARTE 3)

col_transformer = ColumnTransformer(
    [
        ("drop_ids", "drop", drop_cols),
        ("numerical", num_std_pipeline, num_cols),
        ("categorical", cat_pipeline, cat_cols)
    ], verbose_feature_names_out=False, remainder="drop"
)

col_transformer.set_output(transform="pandas")


# PIPELINE DEL MODELO (PARTE 4)

pipeline_dummyreg = Pipeline(
    steps=[
        ("date_features", date_transformer),
        ("col_transformer", col_transformer),
        ("regresor_dummy", DummyRegressor(strategy="mean"))
    ]
)

# ENTRENAMIENTO DEL MODELO DUMMYREGRESSOR

pipeline_dummyreg.fit(X_train, y_train)

# PREDICCIONES DUMMYREGRESSOR

y_pred_val_dummyreg = pipeline_dummyreg.predict(X_val)

#### REPORTE DUMMY MAE para dummyregressor (PARTE 5)

print("Valor MAE (DummyRegressor) para conjunto Validación: ",
      MAE(y_val, y_pred_val_dummyreg))

### XGBOOST (PARTE 6)

pipeline_xgb = Pipeline(
    steps=[
        ("date_features", date_transformer),
        ("col_transformer", col_transformer),
        ("regresor_xgb", XGBRegressor(random_state=42))
    ]
)

# ENTRENAMIENTO DEL MODELO XGBOOST REGRESSOR

pipeline_xgb.fit(X_train, y_train)

# PREDICCIONES XGBOOST REGRESSOR

y_pred_val_xgb = pipeline_xgb.predict(X_val)

# reporte de métrica MAE para xgb regressor

print("Valor MAE (XGBRegressor) para conjunto Validación: ",
      MAE(y_val, y_pred_val_xgb))


Valor MAE (DummyRegressor) para conjunto Validación:  12484.902691741254
Valor MAE (XGBRegressor) para conjunto Validación:  2868.03515625


**interpretación de la comparación de los MAE's**

In [6]:
# PARTE 7: Guardado de modelos

# Para dummy_regressor

# Definir carpeta base para dummyregresor
base_path = os.path.join(os.getcwd(), "Laboratorios", "Laboratorio_6", "data")
os.makedirs(base_path, exist_ok=True)

# Guardar predicciones
pd.DataFrame({"predicciones": y_pred_val_dummyreg}).to_csv(
    os.path.join(base_path, "predicciones_dummy_regressor.csv"), index=False
)

# Guardar modelo
with open(os.path.join(base_path, "dummy_regressor_model.sav"), "wb") as f:
    pickle.dump(pipeline_dummyreg, f)

# Cargar modelo
with open(os.path.join(base_path, "dummy_regressor_model.sav"), "rb") as f:
    modelo_loaded = pickle.load(f)

# # Usar modelo cargado
# y_pred_modelo_loaded = modelo_loaded.predict(X_val)

# para xgboost regressor

# Definir carpeta base para xgboost
base_path = os.path.join(os.getcwd(), "Laboratorios", "Laboratorio_6", "data")
os.makedirs(base_path, exist_ok=True)

# Guardar predicciones
pd.DataFrame({"predicciones": y_pred_val_xgb}).to_csv(
    os.path.join(base_path, "predicciones_xgb_regressor.csv"), index=False
)

# Guardar modelo
with open(os.path.join(base_path, "xgb_regressor_model.sav"), "wb") as f:
    pickle.dump(pipeline_xgb, f)

# Cargar modelo
with open(os.path.join(base_path, "xgb_regressor_model.sav"), "rb") as f:
    modelo_loaded = pickle.load(f)


## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [7]:
## Hint 1 y 2: obtener el nombre de las columnas en el
## paso anterior 
## pipeline_xgb.get_feature_names_out() no funciona porque 
## date_trasnformer,  al ser un FunctionTrasnformer, no tiene ese método.

X_train_trans = pipeline_xgb.named_steps["date_features"].transform(X_train)
X_train_trans = pipeline_xgb.named_steps["col_transformer"].fit_transform(X_train_trans)

features_names = pipeline_xgb.named_steps["col_transformer"].get_feature_names_out()
print(features_names)



['lat' 'long' 'pop' 'price' 'city_Athens' 'city_Irakleion' 'city_Larisa'
 'city_Patra' 'city_Thessaloniki' 'shop_shop_1' 'shop_shop_2'
 'shop_shop_3' 'shop_shop_4' 'shop_shop_5' 'shop_shop_6'
 'brand_adult-cola' 'brand_gazoza' 'brand_kinder-cola' 'brand_lemon-boost'
 'brand_orange-power' 'container_can' 'container_glass'
 'container_plastic' 'capacity_1.5lt' 'capacity_330ml' 'capacity_500ml'
 'day_28' 'day_29' 'day_30' 'day_31' 'month_1' 'month_2' 'month_3'
 'month_4' 'month_5' 'month_6' 'month_7' 'month_8' 'month_9' 'month_10'
 'month_11' 'month_12' 'year_2012' 'year_2013' 'year_2014' 'year_2015'
 'year_2016']


In [8]:
## ahora se tiene el orden real de las columnas
## todas las columnas que no sean "price", deben llevar 0
## para price debe ser -1

monotone_constraints_mapeo = {name: 0 for name in features_names} # acá se genera un diccionario con 0's 

monotone_constraints_mapeo["price"] = -1 # se pone -1 porque es una relación inversa con quantity
                                            # decreasing constraint

print(monotone_constraints_mapeo)

{'lat': 0, 'long': 0, 'pop': 0, 'price': -1, 'city_Athens': 0, 'city_Irakleion': 0, 'city_Larisa': 0, 'city_Patra': 0, 'city_Thessaloniki': 0, 'shop_shop_1': 0, 'shop_shop_2': 0, 'shop_shop_3': 0, 'shop_shop_4': 0, 'shop_shop_5': 0, 'shop_shop_6': 0, 'brand_adult-cola': 0, 'brand_gazoza': 0, 'brand_kinder-cola': 0, 'brand_lemon-boost': 0, 'brand_orange-power': 0, 'container_can': 0, 'container_glass': 0, 'container_plastic': 0, 'capacity_1.5lt': 0, 'capacity_330ml': 0, 'capacity_500ml': 0, 'day_28': 0, 'day_29': 0, 'day_30': 0, 'day_31': 0, 'month_1': 0, 'month_2': 0, 'month_3': 0, 'month_4': 0, 'month_5': 0, 'month_6': 0, 'month_7': 0, 'month_8': 0, 'month_9': 0, 'month_10': 0, 'month_11': 0, 'month_12': 0, 'year_2012': 0, 'year_2013': 0, 'year_2014': 0, 'year_2015': 0, 'year_2016': 0}


In [9]:
monotone_constraints_mapeo["price"]

-1

In [10]:
# se genera lista con los valores del diccionario de arriba
monotone_constraints = [monotone_constraints_mapeo[name] for name in features_names] # [0, 0, 0, -1, ..., 0]

# ahora se tiene que transformar a tupla

monotone_constraints = tuple(monotone_constraints) # (0, 0, 0, -1, ...., 0)

In [11]:
# Inserte su código acá

### XGBOOST 

pipeline_xgb_wc = Pipeline(
    steps=[
        ("date_features", date_transformer),
        ("col_transformer", col_transformer),
        ("regresor_xgb", XGBRegressor(random_state=42, 
        monotone_constraints = monotone_constraints))
    ]
)

# ENTRENAMIENTO DEL MODELO XGBOOST REGRESSOR

pipeline_xgb_wc.fit(X_train, y_train)

# PREDICCIONES XGBOOST REGRESSOR

y_pred_val_xgb_wc = pipeline_xgb_wc.predict(X_val)

print("Valor MAE (XGB) para conjunto Validación (wc: with constraints): ",
      MAE(y_val, y_pred_val_xgb_wc))

Valor MAE (XGB) para conjunto Validación (wc: with constraints):  2931.6552734375


In [12]:
### Guardado del modelo: _wc -> with constraints

# Definir carpeta base para dummyregresor
base_path = os.path.join(os.getcwd(), "Laboratorios", "Laboratorio_6", "data")
os.makedirs(base_path, exist_ok=True)

# Guardar predicciones
pd.DataFrame({"predicciones": y_pred_val_xgb_wc}).to_csv(
    os.path.join(base_path, "predicciones_xgb_regressor_wc.csv"), index=False
)

# Guardar modelo
with open(os.path.join(base_path, "xgb_regressor_model_wc.sav"), "wb") as f:
    pickle.dump(pipeline_xgb_wc, f)

# Cargar modelo
with open(os.path.join(base_path, "xgb_regressor_model_wc.sav"), "rb") as f:
    modelo_loaded = pickle.load(f)

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [ ]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Inserte su código acá

def objective(trial):
    # Inserte su código acá
    pass

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
#!pip install optuna-integration[xgboost]

In [ ]:
# Inserte su código acá

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [ ]:
# Inserte su código acá

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [ ]:
# Inserte su código acá

# Conclusión
Exito!
<p align="center">
  <img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif">
</p>